In [1]:
import cv2
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [4]:
X = np.load("./npy/train_data28.npy")
#X = pd.DataFrame(X)

In [5]:
train = pd.read_csv("D:\\aptos2019-blindness-detection\\train.csv")
y = train['diagnosis'].values
y = to_categorical(y)

In [6]:
y

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# Feature layer 

In [9]:
'''extractor = keras.Model(inputs=model.inputs,
                        outputs=[layer.output for layer in model.layers])
features = extractor(data)'''

inter_output_model = keras.Model(model.input, model.get_layer(index = 7).output )

# train the model

In [ ]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test),batch_size=32)

Epoch 1/20
37/92 [===========>..................] - ETA: 1s - loss: 1.1439 - accuracy: 0.5743

# To Save


In [ ]:
model.save("1a.h5")

In [ ]:
feat_train = inter_output_model.predict(X_train)

In [ ]:
print(feat_train.shape)

In [ ]:
feat_train

In [ ]:
feat_val = inter_output_model.predict(X_test)

In [ ]:
feat_val ,feat_val.shape

In [ ]:
y_train.shape

# SVM

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf')

svm.fit(feat_train,np.argmax(y_train,axis=1))
print('fitting done !!!')

# using traing data

In [ ]:
svm.score(feat_train,np.argmax(y_train,axis=1))


# using val set data

In [ ]:
svm.score(feat_val,np.argmax(y_test,axis=1))


# prediction on test data

In [ ]:
q = np.load("E:\\be project\\npy\\test_data28.npy")
#q =pd.DataFrame(q)
pre =inter_output_model.predict(q)

In [ ]:
Pred_labels = svm.predict(pre)
Pred_labels

# saving results

In [ ]:
sample = pd.read_csv('E:\\be project\\aptos2019-blindness-detection\\sample_submission.csv')
sample['diagnosis'] = Pred_labels
sample.to_csv('sample_cnnr.csv', header=True, index=False)

In [ ]:
ls

In [ ]:
pd.read_csv('sample_cnnr.csv')